# Markov Process

## Markov Reward Process

In [1]:
import numpy as np
np.random.seed(0)
# define the transition matrix
# this is Markov reward process
P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)

rewards = [-1, -2, -2, 10, 1, 0] # rewards vector
gamma = 0.5 # discounting factor

# compute the gain
def compute_return(start_index, chain, gamma):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = gamma * G + rewards[chain[i] - 1] # ? why minus 1?
    return G

chain = [1, 2, 3, 6]
start_index = 0
G = compute_return(start_index, chain, gamma)
print("Cumulated reward computed from trajectory: %s." % G)

Cumulated reward computed from trajectory: -2.5.


In [2]:
def compute(P, rewards, gamma, state_num):
    '''
    Bellman Equation, solve the exact solution.

    state_num: number of the states.
    '''
    rewards = np.array(rewards).reshape((-1, 1)) # flatten the reward vector
    # solve Bellman Equation
    value = np.dot(np.linalg.inv(np.eye(state_num, state_num) - gamma * P), rewards)

    return value

V = compute(P, rewards, gamma, 6)
print("The value of each state in MRP is: \n", V)


The value of each state in MRP is: 
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


## Markov Decision Process

In [3]:
S = ["s1", "s2", "s3", "s4", "s5"]
A = ["stays1", "towards1", "towards2", "towards3", "towards4", "towards5", "randomwalk"]
P = {
    "s1-stays1-s1": 1.0,
    "s1-towards2-s2": 1.0,
    "s2-towards1-s1": 1.0,
    "s2-towards3-s3": 1.0,
    "s3-towards4-s4": 1.0,
    "s3-towards5-s5": 1.0,
    "s4-towards5-s5": 1.0,
    "s4-randomwalk-s2": 0.2,
    "s4-randomwalk-s3": 0.4,
    "s4-randomwalk-s4": 0.4,
}

R = {
    "s1-stays1": -1,
    "s1-towards2": 0,
    "s2-towards1": -1,
    "s2-towards3": -2,
    "s3-towards4": -2,
    "s3-towards5": 0,
    "s4-towards5": 10,
    "s4-randomwalk": 1,
}

gamma = .5
MDP = (S, A, P, R, gamma)

# policy 1
Pi_1 = {
    "s1-stays1": 0.5,
    "s1-towards2": 0.5,
    "s2-towards1": 0.5,
    "s2-towards3": 0.5,
    "s3-towards4": 0.5,
    "s3-towards5": 0.5,
    "s4-towards5": 0.5,
    "s4-randomwalk": 0.5,
}

# policy 2
Pi_2 = {
    "s1-stays1": 0.6,
    "s1-towards2": 0.4,
    "s2-towards1": 0.3,
    "s2-towards3": 0.7,
    "s3-towards4": 0.5,
    "s3-towards5": 0.5,
    "s4-towards5": 0.1,
    "s4-randomwalk": 0.9,
}

def join(str1, str2):
    return str1 + '-' + str2

gamma = 0.5
# compute the transition matrix
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]

P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]
V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)

print("The value of each state in MDP is: \n", V)

The value of each state in MDP is: 
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


## Monte Carlo Sampling to Compute Value

In [4]:
def sample(MDP, Pi, timestep_max, number):
    '''
    MC sampling:
    Policy: Pi
    Timespan maximal: timestep_max
    Sample size: number
    '''
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        # randomly pick a state except state 5 as the start point.
        s = S[np.random.randint(4)]
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            # ref discrete probability distribution sampling
            rand, temp = np.random.rand(), 0
            # choose the action from the policy defined
            for a_opt in A:
                temp += Pi.get(join(s, a_opt), 0) # why +=
                if temp > rand:
                    a = a_opt
                    r = R.get(join(s, a), 0)
                    break
            rand, temp = np.random.rand(), 0
            # transit to the s_next according to the transition matrix
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:
                    s_next = s_opt
                    break
            episode.append((s, a, r, s_next))
            s = s_next
        episodes.append(episode)
    return episodes

episodes = sample(MDP, Pi_1, 20, 5)
print('The first sequence\n', episodes[0])
print('The second sequence\n', episodes[1])
print('The fifth sequence\n', episodes[4])

The first sequence
 [('s1', 'towards2', 0, 's2'), ('s2', 'towards3', -2, 's3'), ('s3', 'towards5', 0, 's5')]
The second sequence
 [('s4', 'randomwalk', 1, 's4'), ('s4', 'towards5', 10, 's5')]
The fifth sequence
 [('s2', 'towards3', -2, 's3'), ('s3', 'towards4', -2, 's4'), ('s4', 'towards5', 10, 's5')]


In [5]:
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] += 1
            V[s] += (G - V[s]) / N[s]

timestep_max = 20
# sample for 1000 times
episodes = sample(MDP, Pi_1, timestep_max, 1000)
gamma = 0.5
V = {"s1": 0, 's2': 0, 's3': 0, 's4': 0, 's5': 0}
N = {"s1": 0, 's2': 0, 's3': 0, 's4': 0, 's5': 0}

MC(episodes, V, N, gamma)
print("The value function of all states estimated by Monte Carlo is: \n", V)

The value function of all states estimated by Monte Carlo is: 
 {'s1': -1.228923788722258, 's2': -1.6955696284402704, 's3': 0.4823809701532294, 's4': 5.967514743019431, 's5': 0}


## State Visitation & Occupancy

In [8]:
def occupancy(episodes, s, a, timestep_max, gamma):
    '''estimate the occupancy of state s at time t using episodes'''
    rho = 0
    total_times = np.zeros(timestep_max)
    occur_times = np.zeros(timestep_max)
    for episode in episodes:
        for i in range(len(episode)):
            (s_opt, a_opt, r, s_next) = episode[i]
            total_times[i] += 1
            if s_opt == s and a_opt == a:
                occur_times[i] += 1
    for i in reversed(range(timestep_max)):
        if total_times[i]:
            rho += gamma ** i * occur_times[i] / total_times[i]

    return rho * (1 - gamma)

gamma = 0.5
timestep_max = 1000
episodes_1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000)
rho_1 = occupancy(episodes_1, 's4', 'randomwalk', timestep_max, gamma)
rho_2 = occupancy(episodes_2, 's4', 'randomwalk', timestep_max, gamma)
print(rho_1, rho_2)

0.11365220293209718 0.2487246201875158
